In [1]:
import numpy as np
import pandas as pd
from pandas import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests 
import time
import re

# pip install geopy
from geopy.geocoders import Nominatim

import warnings
warnings.filterwarnings("ignore")

from geopy_functions import *
from my_functions import *

In [2]:
# import the dataframes
df_uk_masters = pd.read_csv('Datasets/df_uk_masters.csv')                         # all the albums from the UK
df_us_masters = pd.read_csv('Datasets/df_us_masters.csv')                         # albums from the US until 1996, 1998 and 2000
df_us_new_masters = pd.read_csv('Datasets/df_us_new_masters.csv')                         # albums from the US from 1997, 1999 and 2001
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv', keep_default_na=False)  # albums with >= 20 votes, mostly from rock, worldwide
df_masters_blended = pd.read_csv('Datasets/df_masters_blended.csv')               # albums from the UK and US (and others) with >= 10 votes 

# print information
print(f'{df_uk_masters.shape}: df_uk_masters')
print(f'{df_us_masters.shape}: df_us_masters')
print(f'{df_us_new_masters.shape}: df_us_new_masters')
print(f'{df_ratings_20.shape}: df_ratings_20')
print(f'{df_masters_blended.shape}: df_masters_blended')

(56660, 13): df_uk_masters
(48690, 13): df_us_masters
(3641, 13): df_us_new_masters
(51222, 5): df_ratings_20
(11920, 13): df_masters_blended


## Coses pendents, artistes a revisar

Hi ha dos grups que es diuen 'Embrace', un de UK i un de USA

**YOB** te **8** albums a Sputnik ``df_ratings_20``, però nomes **2** a Discogs ``df_masters_blended``, tot i que és una banda de UK, ja que no hi ha releases a UK d'aquests 6 altres àlbums 

Si apareixen al df de US, hauré de fer un concat/merge

In [6]:
df_masters_blended[df_masters_blended['artist']=='YOB']

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
2241,563922,729329,6054020,US,YOB,Clearing The Path To Ascend,2014,62.53,4,['Album'],['Rock'],"['Doom Metal', 'Sludge Metal']",Yob is an American stoner/doom metal band from...
5833,563922,129480,2806549,US,YOB,Catharsis,2003,49.07,3,['Album'],['Rock'],['Doom Metal'],Yob is an American stoner/doom metal band from...


In [142]:
df_ratings_20[df_ratings_20['artist']=='YOB']

,album_id,artist,title,rating,votes
2485,3646,YOB,The Illusion Of Motion,3.94,183
15507,29146,YOB,The Unreal Never Lived,4.06,225
19335,39606,YOB,The Great Cessation,3.90,205
24401,60240,YOB,Elaborations Of Carbon,3.42,73
24402,60241,YOB,Catharsis,3.99,140
28092,80046,YOB,Atma,3.78,259
36277,163668,YOB,Clearing The Path To Ascend,3.92,331
43372,282109,YOB,Our Raw Heart,3.87,295


# **Locations Wikipedia scraping**

## Import dataframes

In [9]:
df = pd.read_csv('Datasets/df_rock_ratings.csv', keep_default_na=False)

print(f"{df['artist'].nunique()} artists")
print(f"{df.shape[0]} albums")
print(f"Average of {round(df.shape[0] / df['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 10 votes")

FileNotFoundError: [Errno 2] No such file or directory: 'Datasets/df_rock_ratings.csv'

In [9]:
df_20 = pd.read_csv('Datasets/df_rock_ratings_20.csv', keep_default_na=False)

print(f"{df_20['artist'].nunique()} artists")
print(f"{df_20.shape[0]} albums")
print(f"Average of {round(df_20.shape[0] / df_20['artist'].nunique(), 2)} albums per artist in the subset with the (mostly UK) albums with more than 20 votes")

3823 artists
9380 albums
Average of 2.45 albums per artist in the subset with the (mostly UK) albums with more than 20 votes


In [ ]:
df_ratings_20 = pd.read_csv('Datasets/df_ratings_20.csv')
df_ratings_20.shape

(51252, 5)

## Testing

In [28]:
df.sample(5)

,artist,title,year,album_length,tracks,genres,styles,release_country,artist_profile
10580,The 1975,A Brief Inquiry into Online Relationships,2018,58.43,15,"['Rock', 'Pop']","['Indie Rock', 'Alternative Rock', 'Indie Pop']",UK & Europe,British indie rock band. \r\n\r\nPop-rock band...
9889,Le Butcherettes,A Raw Youth,2015,0.00,12,['Rock'],[],US,Formed by Teri Gender Bender and Auryn Jolene ...
6394,John Fogerty,Centerfield,1985,35.33,9,['Rock'],"['Pop Rock', 'Folk Rock', 'Country Rock']",US,"American musician, songwriter, and guitarist (..."
4498,L7,The Beauty Process: Triple Platinum,1997,41.57,12,['Rock'],"['Punk', 'Grunge']",US,American grunge punk/alternative rock band fro...
5742,The Fall,Are You Are Missing Winner,2001,47.68,10,['Rock'],"['Garage Rock', 'Punk', 'Rockabilly']",UK,"Post-punk band from Greater Manchester, UK. 19..."


In [125]:
artists = df['artist'].unique()
len(artists)

9616

In [106]:
artists[4155]

'Life at These Speeds'

In [116]:
artist = "AABSINTHE"
name_changed = artist.title().replace(' ', '_')
name_changed

'Aabsinthe'

In [658]:
artist = "John Grant"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')
name_changed_musician = name_changed + ('_(musician)')

url = f"https://en.wikipedia.org/wiki/{name_changed_musician}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the musician is dead
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Genres" in text:
                        location = text.split('Genres')[0]
                        print('Genres')
                    else:
                        if "Occupations" in text:
                            location = text.split('Occupations')[0]
                            print('Occupations')
                        else:
                            location = np.nan
        except:  
            location = np.nan
except:
    print('fuck')


Genres


In [659]:
location

'Buchanan, Michigan, U.S.'

In [652]:
name_changed_musician

'John_Grant_(musician)'

In [533]:
artist = "Stone Gossard"
name_changed = artist.replace(' ', '_')
name_changed_band = artist.replace(' ', '_') + ('_(band)')

url = f"https://en.wikipedia.org/wiki/{name_changed}"
response = requests.get(url).content
soup = BeautifulSoup(response, "html.parser")
table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

try:
    location = table[0].text.split('Origin')[1].split('Genres')[0]

# save info in lists
    print('origin')

except:
    text = table[0].text

    # Step 1: Extract the part after 'Born'
    after_born = text.split("Born", 1)[1]

    text_age = re.search("aged", after_born)

    if text_age:
        # This means the artist is dead
        print('dead')
        location = re.split(r'(19\d{2})', after_born)[4].split('Died')[0].strip()
    else:
        try:
            text = re.split(r'(19\d{2})', after_born)[4].split(')')[1]

            if "Other\xa0names" in text:
                location = text.split('Other\xa0names')[0]
            else:
                if "Citizenship" in text:
                    location = text.split('Citizenship')[0]
                else:
                    if "Occupations" in text:
                        location = text.split('Occupations')[0]
                    else:
                        if "Genres" in text:
                            location = text.split('Genres')[0]
                            print(repr(location))
                        else:
                            location = np.nan
        except:  
            location = np.nan


In [535]:
text

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metalOccupationsMusiciansongwriterInstrumentsGuitarvocalsYears active'

In [536]:
location

'Seattle, Washington, U.S.GenresAlternative rockgrungeglam punkpunk rockhard rockheavy metalglam metal'

In [118]:
df.head()

,album_id,artist,album,rating,votes
0,37,Less Than Jake,Losing Streak,3.90,414
1,40,Sparta,Wiretap Scars,3.79,431
2,41,Muse,Absolution,3.99,4411
3,42,Muse,Showbiz,3.50,2181
4,45,Finch,What It Is to Burn,3.69,864


## **Changing dataframes**

### **``df_artists_origins``**

In [93]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
df_artists_origins.shape

(3251, 2)

In [94]:
df_artists_origins.head()

,artist,origin
0,Sparta,"El Paso, Texas"
1,Muse,"Teignmouth, Devon, England"
2,Finch,"Temecula, California, Estados Unidos"
3,Transplants,"Los Angeles, California, United States[1]"
4,Rooney,"Los Angeles, California, U.S."


In [95]:
df_artists_origins[df_artists_origins['origin']=='United Kingdom']

,artist,origin


In [86]:
df_new_artists = df_artists_origins[df_artists_origins['origin']=='United Kingdom']

In [89]:
df_new_artists

,artist,origin
1059,Son of Dork,"London, England"
1192,Mojave 3,"London, England"
1326,Crippled Black Phoenix,"Bristol, England"
1597,This Mortal Coil,"Wandsworth, England"
2191,Arcadia,"London, England"
2300,Jade Warrior,"London, England"
2348,The Waterboys,"London, England"
2469,Blackmore's Night,"Mount Sinai, NY, United States"
2556,Atomic Rooster,"London, England"
2609,The Nefilim,"Lambeth, England"


In [88]:
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Son of Dork", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Mojave 3", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Crippled Black Phoenix", "Bristol, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="This Mortal Coil", "Wandsworth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Arcadia", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Jade Warrior", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Waterboys", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Blackmore's Night", "Mount Sinai, NY, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Atomic Rooster", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Nefilim", "Lambeth, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Black Spiders", "Sheffield, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Brontide", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Gilgamesh", "Hampstead, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Young Legionnaire", "London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="The Deviants", "Ladbroke Grove, London, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Head of David", "Dudley, West Midlands, United Kingdom", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Quintessence", "London, England", df_new_artists["origin"])

In [53]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[8016]['artist_profile']
splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

Formed in 1969, they played a blend of jazz, progressive rock, Indian Music, and new age rock.   Members included:
Sambhu Babaji : Bass  Dave Codling : Guitar  Shiva Shankar Jones : Keyboards, Vocals  Jake Milton : Drums  Alan Mostert
: Guitar  Raja Ram : Flute, Piano, Vocals


In [52]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Quintessence".strip()].sort_values('votes', ascending=False).head()

,year,artist,title,rating,votes,album_length,tracks,styles,release_country,artist_profile
8016,1969,Quintessence,In Blissful Company,3.88,16,0.0,8,['Psychedelic Rock'],UK,"Formed in 1969, they played a blend of jazz, p..."


In [90]:
df_artists_origins = df_artists_origins[df_artists_origins['origin']!='United Kingdom']
df_artists_origins.shape

(3234, 2)

In [91]:
df_artists_origins.to_csv('Datasets/df_artists_origins.csv', index=False)

In [92]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3251, 2)


In [56]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/9 - London, Greater London, England, United Kingdom
2/9 - Bristol, City of Bristol, West of England, England, United Kingdom
3/9 - Wandsworth, London Borough of Wandsworth, London, Greater London, England, SW18 1UJ, United Kingdom
4/9 - Mount Sinai, Miller Place, Town of Brookhaven, Suffolk County, New York, 11766, United States
5/9 - Lambeth, London Borough of Lambeth, London, Greater London, England, SE1 7JW, United Kingdom
6/9 - Sheffield, South Yorkshire, England, United Kingdom
7/9 - Hampstead, Greater London, England, NW3 1QG, United Kingdom
8/9 - Ladbroke Grove, Westway, Lancaster West Estate, North Kensington, Royal Borough of Kensington and Chelsea, London, Greater London, England, W10 5YG, United Kingdom
9/9 - Dudley, West Midlands, England, United Kingdom


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Bristol,"Bristol, England","Bristol, England",51.453802,-2.597298,"Bristol, City of Bristol, West of England, Eng..."
1,United Kingdom,Dudley,"Dudley, West Midlands, United Kingdom","Dudley, West Midlands, United Kingdom",52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
2,United Kingdom,Hampstead,"Hampstead, England","Hampstead, England",51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3,United Kingdom,Ladbroke Grove,"Ladbroke Grove, London, England","Ladbroke Grove, London, England",51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
4,United Kingdom,Lambeth,"Lambeth, England","Lambeth, England",51.495211,-0.116335,"Lambeth, London Borough of Lambeth, London, Gr..."
5,United Kingdom,London,"London, England","London, England",51.507446,-0.127765,"London, Greater London, England, United Kingdom"
6,United Kingdom,Sheffield,"Sheffield, England","Sheffield, England",53.380663,-1.470228,"Sheffield, South Yorkshire, England, United Ki..."
7,United Kingdom,Wandsworth,"Wandsworth, England","Wandsworth, England",51.457027,-0.193261,"Wandsworth, London Borough of Wandsworth, Lond..."
8,United States,Mount Sinai,"Mount Sinai, NY, United States","Mount Sinai, NY, United States",40.941066,-73.019455,"Mount Sinai, Miller Place, Town of Brookhaven,..."


### **``df_coordinates``**

In [57]:
df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
df_coordinates_scraped.shape

(1527, 7)

In [58]:
df_coordinates_scraped.head()

,country,city,origin,origin_clean,latitude,longitude,address
0,Australia,Adelaide,"Adelaide, South Australia, Australia","Adelaide, South Australia, Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
1,Australia,Adelaide,"Adelaide, South Australia","Adelaide, South Australia",-34.928181,138.599931,"Adelaide, Adelaide City Council, South Austral..."
2,Australia,Blue Mountains,"Blue Mountains, NSW, Australia","Blue Mountains, NSW, Australia",-33.609741,150.405224,"Blue Mountains, New South Wales, Australia"
3,Australia,Brisbane,"Brisbane, Queensland, Australia","Brisbane, Queensland, Australia",-27.468968,153.023499,"City of Brisbane, Queensland, Australia"
4,Australia,Byron Bay,"Byron Bay, New South Wales, Australia","Byron Bay, New South Wales, Australia",-28.648333,153.617778,"Byron Bay, Byron Shire Council, New South Wale..."


In [59]:
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address
702,United Kingdom,United Kingdom,United Kingdom,United Kingdom,54.702354,-3.276575,United Kingdom


In [60]:
df_coordinates_scraped.drop(702, axis=0, inplace=True)
df_coordinates_scraped[df_coordinates_scraped['city']=='United Kingdom']

,country,city,origin,origin_clean,latitude,longitude,address


In [62]:
df_coordinates_scraped.to_csv('Datasets/df_coordinates.csv', index=False)

In [63]:
df_coordinates.shape

(9, 7)

In [64]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1526, 7)

Found 4 duplicates:
               city         country
324         Bristol  United Kingdom
515  Ladbroke Grove  United Kingdom
542          London  United Kingdom
650       Sheffield  United Kingdom

Resulting dataset: (1531, 7)
Merged artists with coordinates! Found 5 new locations
df_coordinates_concat exported to .csv


### **``df_artists_origins_coordinates_concat``**

In [74]:
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3246,Gilgamesh,United Kingdom,Hampstead,51.556530,-0.178301,"Hampstead, Greater London, England, NW3 1QG, U..."
3247,Young Legionnaire,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"
3248,The Deviants,United Kingdom,Ladbroke Grove,51.517264,-0.211102,"Ladbroke Grove, Westway, Lancaster West Estate..."
3249,Head of David,United Kingdom,Dudley,52.511083,-2.081681,"Dudley, West Midlands, England, United Kingdom"
3250,Quintessence,United Kingdom,London,51.489334,-0.144055,"London, Greater London, England, United Kingdom"


## **Changing values**

In [72]:
df_masters_blended[df_masters_blended['artist'].str.contains('Nucleus')]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
1004,3821235,1537522,13529787,US,Nucleus (US),Entity,2019,38.42,8,['Album'],['Rock'],['Death Metal'],"Death Metal band from Chicago, Illinois, USA. ..."
1592,3821235,1094310,8362817,US,Nucleus (US),Sentient,2016,37.92,9,['Album'],['Rock'],['Death Metal'],"Death Metal band from Chicago, Illinois, USA. ..."
5257,184256,175620,279855,UK,Nucleus (UK),We'll Talk About It Later,1971,0.00,7,"['LP', 'Album']","['Jazz', 'Rock']","['Fusion', 'Jazz-Funk', 'Jazz-Rock', 'Prog Rock']","Pioneering jazz-rock, progressive, psychedelic..."
10195,184256,23574,465143,UK,Nucleus (UK),Elastic Rock,1970,0.00,13,"['LP', 'Album']","['Jazz', 'Rock']","['Jazz-Rock', 'Fusion', 'Prog Rock']","Pioneering jazz-rock, progressive, psychedelic..."


In [75]:
df_masters_blended[df_masters_blended['title'].str.contains('Alleycat')]

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile


In [77]:
df_ratings_20[df_ratings_20['artist'].str.contains('Nucleus')]

,album_id,artist,title,rating,votes
29004,86398,Nucleus (UK),Elastic Rock,3.55,20
29120,87140,Nucleus (UK),We'll Talk About It Later,3.79,21
39636,216509,Nucleus (US),Sentient,3.29,29
45524,333839,Nucleus (US),Entity,3.68,40
46380,352917,Nucleus (UK),Alleycat,3.52,20


In [76]:
df_ratings_20.loc[46380, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[29004, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[29120, 'artist'] = 'Nucleus (UK)'
df_ratings_20.loc[39636, 'artist'] = 'Nucleus (US)'
df_ratings_20.loc[45524, 'artist'] = 'Nucleus (US)'

In [869]:
df.drop([7660, 8037], axis=0, inplace=True)
df.shape

(12059, 10)

In [122]:
df['artist'] = np.where(df['artist']=='pg.99 / Majority Rule', 'Majority Rule', df['artist'])
df[df['artist']=='pg.99 / Majority Rule']

,album_id,artist,album,rating,votes


In [78]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [581]:
df.shape

(12059, 10)

### **Testing code for strange cases**

In [ ]:
df = pd.read_csv('Datasets/df_ratings.csv')
artists = df['artist'].unique()

artists_list = []
origin_list = []
count=0

for index in artists[140:150]:

    artists_list.append(index)
    name_changed = index.replace(' ', '_')
    name_changed_band = name_changed + ('_(band)')

    try:
        url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
        response = requests.get(url).content
        soup = BeautifulSoup(response, "html.parser")

        origin = soup.select('table tr th', class_='infobox-label')

        if len(origin) > 0:
            try:
                if origin[2].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[1].text
                elif origin[3].text == 'Origin':
                    location = soup.select('table tr td', class_='infobox-data')[2].text
                # else:
                    
                if 'multiple issues' in location:
                    location = soup.select('table tr td', class_='infobox-data')[7].text        
                    print(f'{count} - {name_changed_band}: multiple issues - {location}')
                    origin_list.append(location)
                elif 'additional citations' in location:
                    location = soup.select('table tr td', class_='infobox-data')[3].text        
                    print(f'{count} - {name_changed_band}: additional citations - {location}')
                    origin_list.append(location)

                else:
                    print(f'{count} - {name_changed_band}: {location}')
                    origin_list.append(location)
            except:
                print(f'{count} - {name_changed_band}: {location}')
                origin_list.append(location)      
        else:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")

                origin = soup.select('table tr th', class_='infobox-label')

                if len(origin) > 0:
                    if origin[2].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[1].text

                        if 'multiple issues' in location:
                            location = soup.select('table tr td', class_='infobox-data')[7].text        
                            print(f'{count} - {name_changed_band}: multiple issues - {location}')
                            origin_list.append(location)
                        elif 'additional citations' in location:
                            location = soup.select('table tr td', class_='infobox-data')[3].text        
                            print(f'{count} - {name_changed_band}: additional citations - {location}')
                            origin_list.append(location)
                        else:
                            print(f'{count} - {name_changed_band}: {location}')
                            origin_list.append(location)

                    elif origin[3].text == 'Origin':
                        location = soup.select('table tr td', class_='infobox-data')[2].text
                        print(f'{count} - {name_changed_band}: {location}')
                        origin_list.append(location) 

                    else:
                        print(f'{count} - {index}: no location found')
                        origin_list.append(np.nan)  
                else:
                    print(f'{count} - {index}: short length')
                    origin_list.append(np.nan)
            except:
                print(f'{count} - {index}: error')
                origin_list.append(np.nan)
    except:
        print(f'{count} - {index}: error')
        origin_list.append(np.nan)

    if len(artists_list) != len(origin_list):
        print('different lengths')
        break

    count+=1

0 - Funeral_for_a_Friend_(band): Bridgend, Wales
1 - Millencolin_(band): multiple issues - Örebro, Sweden
2 - The_Flaming_Lips_(band): Oklahoma City, Oklahoma, U.S.
3 - Feeder_(band): Feeder in 2008
4 - Descendents_(band): Manhattan Beach, California, U.S.
5 - PJ Harvey: no location found
6 - Godsmack_(band): Lawrence, Massachusetts U.S.
7 - Blind_Faith_(band): Ripley, Surrey, England
8 - Van_Halen_(band): Pasadena, California, U.S.
9 - Damageplan_(band): Dallas, Texas, U.S.


## Defining the functions

In [15]:
def get_origins_wikipedia(df, start_index, final_index):
    df = pd.read_csv('Datasets/df_rock_ratings_20.csv')
    artists = df['artist'].unique()

    try:
    # import the DataFrame with the locations whose coordinates I already have
        df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
        print('Bingo! df_coordinates.csv found \n')
    except: 
        print('df_coordinates.csv not found \n')

    artists_list = []
    origin_list = []
    count=0
    scraped=0

    for index in artists_us_to_do[start_index:final_index]:

        name_changed = index.replace(' ', '_')
        name_changed_band = name_changed + ('_(band)')

        try:
            url = f"https://en.wikipedia.org/wiki/{name_changed_band}"
            response = requests.get(url).content
            soup = BeautifulSoup(response, "html.parser")

            table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

            location = table[0].text.split('Origin')[1].split('Genres')[0]
            city = location.split(', ')[0]
            count+=1
            
        # save info in lists
            artists_list.append(index)  
            origin_list.append(location)
            scraped+=1
            print(f'{scraped}/{count} - {name_changed_band}: {location}')

        except:
            try:
                url = f"https://en.wikipedia.org/wiki/{name_changed}"
                response = requests.get(url).content
                soup = BeautifulSoup(response, "html.parser")
                table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')

                try:
                    location = table[0].text.split('Origin')[1].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1 
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed}: {location}')

                except:
                    location = table[0].text.split(')')[2].split('Genres')[0]
                    city = location.split(', ')[0]
                    count+=1

                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (individual): {location}')

            except:
                try:
                    url = f"https://es.wikipedia.org/wiki/{name_changed}"
                    response = requests.get(url).content
                    soup = BeautifulSoup(response, "html.parser")

                    table = soup.select('#mw-content-text > div.mw-content-ltr.mw-parser-output > table.infobox')
                    location = table[0].text.split('Origen\n')[1].split(' Información')[0]
                    city = location.split(', ')[0]
                    count+=1    
    
                # save info in lists
                    artists_list.append(index)  
                    origin_list.append(location)
                    scraped+=1
                    print(f'{scraped}/{count} - {name_changed} (español): {location}')

                except:
                    count+=1
                    print(f'{scraped}/{count} - {index}: error')
                    artists_list.append(index) 
                    origin_list.append(np.nan)

        if len(artists_list) != len(origin_list):
            print('different lengths')
            break

    df_artists_origins = pd.DataFrame({'artist': artists_list
                             , 'origin': origin_list})
    
    return df_artists_origins

In [102]:
def get_new_artists(df_artists_origins):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

    if df_artists_origins['origin'].isna().sum() == 0:        
        print("No null values, but let's take a look just in case there are weird locations")

    else: 
    # take a look at the df with the new artists and make sure there are non null values in origin (when it couldn't find it in Wikipedia)
        print(f'{round(df_artists_origins['origin'].isna().sum() / df_artists_origins.shape[0]*100, 2)} % of nulls')
    
# subset of the new artists I just got, wether there are null values or not
    df_new_artists = df_artists_origins[~df_artists_origins['artist'].isin(df_artists_origins_scraped['artist'].values)]

    print("Here is the dataframe with the new artists, without duplicates")
    return df_new_artists   # so I can take a look at it and then continue

In [ ]:
def export_artists_origins_concat(df_new_artists):

# import the df with the artists' origins already scraped
    df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')

# concat with the df I just got
    df_artists_origins_concat = pd.concat([df_artists_origins_scraped, df_new_artists])
    df_artists_origins_concat.drop_duplicates(inplace=True)     # just in case
    df_artists_origins_concat.reset_index(drop=True, inplace=True)

# export all the artists and their origins to a .csv file (the ones I got plus the new artists)
    df_artists_origins_concat.to_csv('Datasets/df_artists_origins.csv', index=False)
    print('df_artists_origins_concat exported to .csv')
    print(df_artists_origins_concat.shape)

In [175]:
def get_coordinates_geopy(df_new_artists):
    
# replace special characters for spaces
    df_new_artists['origin_clean'] = df_new_artists['origin'].str.replace('.', '')
    df_new_artists['origin_clean'] = df_new_artists['origin_clean'].str.replace(r'\[\d+\]', '', regex=True)

# run the function that gets the coordinates from the origins from Geopy
    geolocator = Nominatim(user_agent="music_analysis", timeout=10)

# if they are 'dirty' origins that after the cleaning, they result in the same 'origin_clean'
    df_unique = df_new_artists[['origin', 'origin_clean']].drop_duplicates() 
    unique_origins = df_unique['origin'].values
    unique_origins_clean = df_unique['origin_clean'].values

    country_list = []
    city_list = []
    latitude_list = []
    longitude_list = []
    address_list = []
    lists = [country_list, city_list, latitude_list, longitude_list, address_list]
    count = 0

    for origin in unique_origins_clean:
        count+=1
        time.sleep(1)
        location = geolocator.geocode(origin)

        print(f'{count}/{len(unique_origins_clean)} - {location.address}')  

    # save the info in lists
        country_list.append(location.address.split(', ')[-1])
        city_list.append(origin.split(', ')[0])
        latitude_list.append(location.latitude)
        longitude_list.append(location.longitude)
        address_list.append(location.address)

        # # Check lengths
        # print(f"{count}/{len(unique_origins_clean)} - {origin}")
        # print(f"Current list lengths -> country: {len(country_list)}, city: {len(city_list)}, "
        #     f"lat: {len(latitude_list)}, lon: {len(longitude_list)}, address: {len(address_list)}")

    df_coordinates = pd.DataFrame({'country': country_list
                                , 'city': city_list
                                , 'origin': unique_origins
                                , 'origin_clean': unique_origins_clean
                                , 'latitude': latitude_list
                                , 'longitude': longitude_list
                                , 'address': address_list})
    df_coordinates.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates.reset_index(drop=True, inplace=True)

    return df_coordinates

In [ ]:
def export_coordinates_concat(df_coordinates):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_scraped = pd.read_csv('Datasets/df_coordinates.csv')
    print(f"df_coordinates_scraped: {df_coordinates_scraped.shape}\n")

# concat with the df of the coordinates I just got
    df_coordinates_concat = pd.concat([df_coordinates_scraped, df_coordinates])
    df_coordinates_concat.sort_values(['country', 'city'], inplace=True) # sort by country and city
    df_coordinates_concat.reset_index(drop=True, inplace=True)

# look for duplicates in the origin, between the locations I had already gotten and the new ones
    check_duplicates_origins(df_coordinates_concat)
    new_origins = df_coordinates_concat.shape[0] - df_coordinates_scraped.shape[0]
    print(f"Merged artists with coordinates! Found {new_origins} new locations")

# save it in a csv file (the coordinates I had plus the ones from the new artists I just got)
    df_coordinates_concat.to_csv('Datasets/df_coordinates.csv', index=False)
    print('df_coordinates_concat exported to .csv')

In [140]:
def merge_origins_coordinates(df_new_artists):

# import the last df that contains the coordinates of the unique origins
    df_coordinates_concat = pd.read_csv('Datasets/df_coordinates.csv')

# merge with the previous df with the artists
    df_artists_origins_coordinates = pd.merge(df_new_artists, df_coordinates_concat, on=['origin'])
    df_artists_origins_coordinates.drop(columns=['origin', 'origin_clean_x', 'origin_clean_y'], inplace=True)

# import the df that contains info of the artists and the coordinates of their origins
    df_artists_origins_coordinates_scraped = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')

# concat to get the df with all the artists, origins and their coordinates
    df_artists_origins_coordinates_concat = pd.concat([df_artists_origins_coordinates_scraped, df_artists_origins_coordinates])
    df_artists_origins_coordinates_concat.reset_index(drop=True, inplace=True)

# save it in a csv file
    df_artists_origins_coordinates_concat.to_csv('Datasets/df_artists_origins_coordinates.csv', index=False)
    print("Exported to a .csv file")

    return df_artists_origins_coordinates_concat

## Merge dataframes and look for the ``new_artists``

In [11]:
artists_blend = df_masters_blended['artist'].unique()
len(artists_blend)

4527

In [ ]:
df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists = df_artists_origins['artist'].unique()
artists_usa = []

for artist in artists_blend:
    if artist not in df_artists_origins['artist'].values:
        artists_usa.append(artist)

len(artists_usa)

1555

In [ ]:
artists_usa

['As Living Arrows',
 'Hidden Mothers',
 'Tiny Moving Parts',
 'Poppy',
 'State Champs',
 'Oso Oso',
 'Better Lovers',
 'Lowen',
 'Halsey',
 'Amyl and the Sniffers',
 'Delta Sleep',
 'High Vis',
 'Cemetery Skyline',
 'Goat',
 'Chat Pile',
 'Drug Church',
 'Origami Angel',
 'Heriot',
 'Nightwish',
 'Foxing',
 'Alora Crucible',
 'Wage War',
 'TURQUOISEDEATH',
 'Dawn Treader',
 'Boston Manor',
 'MJ Lenderman',
 'Fat Dog',
 'Kingcrow',
 'Leprous',
 'Wunderhorse',
 'thrown',
 'Horse Jumper of Love',
 'Within the Ruins',
 'Magdalena Bay',
 'Fontaines D.C.',
 'beabadoobee',
 'State Faults',
 'Graphic Nature',
 'The Home Team',
 'Speed',
 'Remi Wolf',
 'Clairo',
 'Crippling Alcoholism',
 'Cigarettes After Sex',
 'Abriction',
 'Pijn',
 'Outlander',
 'Imagine Dragons',
 'The Dangerous Summer',
 'Pond',
 'Hyperdontia',
 'Vredehammer',
 'Pedro the Lion',
 'Weston Super Maim',
 'Stand Still',
 'Mortal Wound',
 'Eye of Solitude',
 'Beth Gibbons',
 'Contention',
 'Knocked Loose',
 'Mdou Moctar',
 'Th

In [110]:
# I have to import this df for the function to properly work
df = pd.read_csv('Datasets/df_blend_ratings.csv')
artists = df['artist'].unique()
len(artists)

4249

In [11]:
artists[-1]

'As Living Arrows'

In [12]:
df_artists_origins.index[-1]

3423

In [13]:
df_artists_origins.tail()

,artist,origin
3419,Cinderella,"Philadelphia, Pennsylvania, U.S."
3420,Death From Above 1979,"Toronto, Ontario, Canada"
3421,Chicago,"Chicago, Illinois, United States"
3422,The Nation of Ulysses,"Washington, D.C."
3423,Strung Out,"Simi Valley, California, U.S."


In [164]:
np.where(artists=='July Talk')

(array([2772], dtype=int64),)

In [163]:
artists[2772]

'July Talk'

## **Code to execute the functions from ``geopy_functions.py``**

In [3]:
artists_to_remove = ['Cemetery Skyline', 'Goat', 'Kingcrow', 'Speed', 'Hyperdontia', 'Vredehammer', 'Weston Super Maim',
                'Mdou Moctar', 'AVRALIZE', 'Engulfed', 'Coffin Storm', 'samlrc', 'Little Kid', 'Termina', 'Rorcal',
                'Reverence To Paroxysm', 'Voyager', 'Blindfolded and Led to the Woods', 'Mork', 'Yeule', 'Pond',
                'Empire State Bastard', 'Blood Command', 'Avatar', 'Brutus', 'Faceless Burial', 'Within Destruction',
                'Sedimentum', 'Ataraxy', 'Corpsessed', 'Epitaphe', 'Aldous Harding', 'Messa', 'Ghost', 'Ecchymosis',
                'Wormrot', 'Vertebra Atlantis', 'Papangu', 'Sermon of Flames', 'Springtime', 'Monolord', 'Sulphurous',
                'LLNN', 'Trna', 'Slaughter To Prevail', 'sonhos tomam conta', 'Galvanizer', 'Ophidian I', 'Diabolizer',
                'Morbific', 'Defacement', 'Divide And Dissolve', 'Soen', 'Scorpions', 'Accept', 'Respire', 'Undergang',
                'Bearings', 'Scalp', 'Miasmatic Necrosis', 'Teenage Mutant Ninja Turtles']

In [4]:
df_masters_blended = pd.read_csv('Datasets/df_masters_blended.csv')
artists_blend = df_masters_blended['artist'].unique()

df_artists_origins = pd.read_csv('Datasets/df_artists_origins.csv')
artists = df_artists_origins['artist'].unique()
artists_usa = []

for artist in artists_blend:
    if artist not in df_artists_origins['artist'].values and artist not in artists_to_remove:
        artists_usa.append(artist)

len(artists_usa)

1132

In [5]:
# show the first next 5 artists I'm going to scrape
artists_usa[:5]

['Matt Berninger',
 'Gorephilia',
 'Joji',
 'Corey Taylor',
 'Fires in the Distance']

In [6]:
# create the df with the origins scraped from Wikipedia

df = pd.read_csv('Datasets/df_masters_blended.csv')
start_index = 0
final_index = start_index+10

df_artists_origins = get_origins_wikipedia(df, start_index, final_index)

Bingo! df_coordinates.csv found 

1/1 - Matt_Berninger (individual): Cincinnati, Ohio, U.S.GenresAlternative rock
1/2 - Gorephilia: error
2/3 - Joji: Higashinada-ku, Kobe, Japan[4][5]
3/4 - Corey_Taylor: Waterloo, Iowa, U.S.
3/5 - Fires in the Distance: error
4/6 - Obsidian_Kingdom: Barcelona, Spain
5/7 - Svalbard_(band): Bristol, England, UK
6/8 - The_Ocean_(band): Berlin, Germany
7/9 - Into_It._Over_It.: Chicago, Illinois, U.S.
7/10 - Fawn Limbs: error


In [26]:
# create a subset of the new artists I just got, and tell me if there are nulls
df_new_artists = get_new_artists(df_artists_origins)

# show the first new artists, if they were duplicates they have been dropped
df_new_artists.head()

9 nulls (45.0 %)


,artist,origin
0,Parquet Courts,"New York City, U.S."
1,Pearl Charles,"Los Angeles, California, United States"
2,Respire,NaN
3,Teenage Mutant Ninja Turtles,al workTeenage Mutant Ninja Turtles(1984)Owner...
4,Dominic Fike,"Naples, Florida, U.S.Genres\nAlternative hip h..."


In [27]:
df_new_artists.shape

(20, 2)

### If there are null or weird values

In [28]:
nulls = df_new_artists[df_new_artists['origin'].isna()]
nulls.shape

(9, 2)

In [64]:
df_new_artists[df_new_artists['origin'].isna()].head(10)

,artist,origin


**``np.where`` to replace the values for the real origins**

In [29]:
for artist in nulls['artist'].values:
    print(f'df_new_artists["origin"] = np.where(df_new_artists["artist"]=="{artist}", "England", df_new_artists["origin"])')

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Respire", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Undergang", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Edenic Past", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Palm Reader", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bearings", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Scalp", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Kingdom of Giants", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Guitar Fight from Fooly Cooly", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Miasmatic Necros

In [46]:
# looking in the internet for the real origins of these artists

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Edenic Past", "New York, NY, United States", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Palm Reader", "Nottingham, England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Kingdom of Giants", "Sacramento, USA", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Guitar Fight from Fooly Cooly", "Cleveland, TN, United States", df_new_artists["origin"])

df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Dominic Fike", "Naples, Florida, U.S.", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])
df_new_artists["origin"] = np.where(df_new_artists["artist"]=="Bomrowmgvo", "England", df_new_artists["origin"])

In [45]:
# look for the albums of the artist in the original df to check it's the correct artist
df[df['artist']=="Teenage Mutant Ninja Turtles	".strip()].sort_values('year').head()

,artist_id,master_id,main_release_id,release_country,artist,title,year,album_length,tracks,release_type,genres,styles,artist_profile
685,1933733,1872192,2430722,US,Teenage Mutant Ninja Turtles,Coming Out Of Their Shells,1990,0.0,10,['Album'],"['Rock', ""Children's""]","['Surf', 'Classic Rock', 'Rock & Roll']",[b]This entry is for all officially licensed s...


In [57]:
# check if there's info of the artist origin in the column 'artist_profile'
import textwrap
artist_profile = df.loc[659]['artist_profile']

splitted_string = textwrap.fill(artist_profile, width=120)
print(splitted_string)

American singer/songwriter.  Sister of [a3575264] and [a5678463].  Daughter of [a5678464]


#### Changing ``df_ratings_20`` in case needed

In [190]:
df_ratings_20.loc[8662, 'artist'] = 'Embrace (US)'
df_ratings_20.loc[4493, 'artist'] = 'Embrace (UK)'
df_ratings_20.loc[4494, 'artist'] = 'Embrace (UK)'

In [ ]:
df_ratings_20.to_csv('Datasets/df_ratings_20.csv', index=False)

In [214]:
df_new_artists['origin'].isna().sum()

0

#### **Drop artists that are not from the UK or the US**

In [ ]:
artists_to_remove = []

for artist in artists_to_remove:
    try:
        artists_usa.remove(artist)
        print(f'{artist} removed')
    except:
        pass

Teenage Mutant Ninja Turtles removed


In [48]:
# I can drop single rows or I can just create a subset with the artists that I want to keep, the ones that are not in the list of artists to remove
df_new_artists = df_new_artists[~df_new_artists['artist'].isin(artists_to_remove)]

In [ ]:
# if they are not brittish or american, I can drop them
print(df_new_artists.shape)
# df_new_artists.drop([4], axis=0, inplace=True)
print(df_new_artists.shape)

(80, 2)
(79, 2)


#### **Check long and short origins, probably wrong**

In [49]:
start_index = 3

end_index = start_index+5
df_new_artists.loc[start_index:end_index]

,artist,origin
4,Dominic Fike,"Naples, Florida, U.S."
6,Edenic Past,"New York, NY, United States"
7,Red City Radio,"Oklahoma City, Oklahoma, United States"
8,Palm Reader,"Nottingham, England"


In [50]:
# print abnormally long origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) > 45:
        print(index, row['origin'])

In [42]:
# print abnormally short origins and visually check if they are correct
for index, row in df_new_artists.iterrows():
    if len(row['origin']) < 10:
        print(index, row['origin'])

### **Changing easy values: individuals that didn't get the right origin in Wikipedia**

In [78]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains('Genres')]

,artist,origin
56,Torres,"Orlando, Florida, USGenresIndie rock"


In [79]:
# individuals that didn't get the right origin in Wikipedia
df_new_artists[df_new_artists['origin'].str.contains(' and ')]

,artist,origin
37,Sam Fender,"North Shields, Tyne and Wear, England"
39,Full of Hell,"Ocean City, Maryland and Pennsylvania, U.S."
70,Nexilva,"Sunderland, Tyne and Wear, United Kingdom"


In [80]:
# bands from Cumbria, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Cumbria')]

,artist,origin


In [81]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace('Cumbria', 'Cumberland'))

In [82]:
# bands from Middlesex, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Middlesex')]

,artist,origin


In [83]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Middlesex', ''))

In [84]:
# bands from Yorkshire, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('West Riding of Yorkshire')]

,artist,origin


In [85]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', West Riding of Yorkshire', ''))

In [86]:
# bands from Merseyside, Geopy doesn't detect it
df_new_artists[df_new_artists['origin'].str.contains('Merseyside')]

,artist,origin


In [87]:
df_new_artists['origin'] = df_new_artists['origin'].apply(lambda x: x.replace(', Merseyside', ''))

In [88]:
# bands from United Kingdom, wrong origin, poor level of detail
df_new_artists[df_new_artists['origin']==('United Kingdom')]

,artist,origin


### **``np.where`` to replace the values for the real origins**

In [ ]:
# looking on the internet for the real origins of these artists

df_new_artists['origin'] = np.where(df_new_artists['artist']=="Hammerhead", "Moorhead, Minnesota, United States", df_new_artists['origin'])

**Try a single origin in Geopy**

In [359]:
# try to get the right origin of an origin that crashed
geolocator = Nominatim(user_agent="music_analysis")

origin = "Long Island, New York, United States"

origin_clean = re.sub(r'\[\d+\]', '', origin).replace('.', '')
location = geolocator.geocode(origin_clean)
print(f"{location.address}")
print(location.latitude, location.longitude)

Long Island, New York, United States
40.85149705 -73.09942966988302


In [99]:
start_index = 78

end_index = start_index+5
df_new_artists[start_index:end_index]

,artist,origin
95,Goat Girl,"London, England"
98,Pom Poko,"Oslo, Norway"
99,The Casket Lottery,"Kansas City, Missouri"


**Try all origins in Geopy**

In [52]:
# try to get the coordinates of the origins from Geopy and see if it crashes (wrong location that I have to change)
geolocator = Nominatim(user_agent="music_analysis", timeout=10)

initial_index = 0
count = initial_index-1

for origin in df_new_artists['origin'].str.replace('.', '').str.replace(r'\[\d+\]', '', regex=True)[initial_index:]:
    count+=1
    location = geolocator.geocode(origin)
    print(f"{count} - {location.address}")

0 - City of New York, New York, United States
1 - Los Angeles, Los Angeles County, California, United States
2 - Naples, Collier County, Florida, United States
3 - City of New York, New York, United States
4 - Oklahoma City, Oklahoma County, Oklahoma, United States
5 - City of Nottingham, Nottinghamshire, East Midlands, England, United Kingdom
6 - Torrance, Los Angeles County, California, United States
7 - Exeter, Devon, England, United Kingdom
8 - Scranton, Lackawanna County, Pennsylvania, United States
9 - Sacramento, Sacramento County, California, United States
10 - Cleveland, Bradley County, East Tennessee, Tennessee, United States
11 - Edmonton, Metcalfe County, Kentucky, 42129, United States
12 - Philadelphia, Philadelphia County, Pennsylvania, United States
13 - Mukilteo, Snohomish County, Washington, United States


## **Export to .csv**

### GeoPy wrong locations

**In case GeoPy fails due to a wrong location, I have to delete the new locations, export again, change the location and run GeoPy again**

In [ ]:
#     # import the df with the artists' origins already scraped
# df_artists_origins_scraped = pd.read_csv('Datasets/df_artists_origins.csv')
# df_artists_origins_scraped = df_artists_origins_scraped[0:-20]
# df_artists_origins_scraped.to_csv('Datasets/df_artists_origins.csv', index=False)

In [53]:
df_new_artists.shape

(14, 2)

In [54]:
# in case I created by mistake already 'origin_clean' and I want to drop it
# df_new_artists = df_new_artists[['artist', 'origin']]
# df_new_artists

### **Export to .csv**

In [55]:
export_artists_origins_concat(df_new_artists)

df_artists_origins_concat exported to .csv
(3981, 2)


## **GeoPy**

In [56]:
df_coordinates = get_coordinates_geopy(df_new_artists)
df_coordinates

1/14 - City of New York, New York, United States
2/14 - Los Angeles, Los Angeles County, California, United States
3/14 - Naples, Collier County, Florida, United States
4/14 - City of New York, New York, United States
5/14 - Oklahoma City, Oklahoma County, Oklahoma, United States
6/14 - City of Nottingham, Nottinghamshire, East Midlands, England, United Kingdom
7/14 - Torrance, Los Angeles County, California, United States
8/14 - Exeter, Devon, England, United Kingdom
9/14 - Scranton, Lackawanna County, Pennsylvania, United States
10/14 - Sacramento, Sacramento County, California, United States
11/14 - Cleveland, Bradley County, East Tennessee, Tennessee, United States
12/14 - Edmonton, Metcalfe County, Kentucky, 42129, United States
13/14 - Philadelphia, Philadelphia County, Pennsylvania, United States
14/14 - Mukilteo, Snohomish County, Washington, United States


,country,city,origin,origin_clean,latitude,longitude,address
0,United Kingdom,Exeter,"Exeter, England","Exeter, England",50.725614,-3.526921,"Exeter, Devon, England, United Kingdom"
1,United Kingdom,Nottingham,"Nottingham, England","Nottingham, England",52.953419,-1.149646,"City of Nottingham, Nottinghamshire, East Midl..."
2,United States,Cleveland,"Cleveland, TN, United States","Cleveland, TN, United States",35.159518,-84.876611,"Cleveland, Bradley County, East Tennessee, Ten..."
3,United States,Edmonton,"Edmonton, Kentucky, U.S.","Edmonton, Kentucky, US",36.980056,-85.612191,"Edmonton, Metcalfe County, Kentucky, 42129, Un..."
4,United States,Los Angeles,"Los Angeles, California, United States","Los Angeles, California, United States",34.053691,-118.242766,"Los Angeles, Los Angeles County, California, U..."
5,United States,Mukilteo,"Mukilteo, Washington, U.S.","Mukilteo, Washington, US",47.947403,-122.303699,"Mukilteo, Snohomish County, Washington, United..."
6,United States,Naples,"Naples, Florida, U.S.","Naples, Florida, US",26.142198,-81.794294,"Naples, Collier County, Florida, United States"
7,United States,New York,"New York, NY, United States","New York, NY, United States",40.712728,-74.006015,"City of New York, New York, United States"
8,United States,New York City,"New York City, U.S.","New York City, US",40.712728,-74.006015,"City of New York, New York, United States"
9,United States,Oklahoma City,"Oklahoma City, Oklahoma, United States","Oklahoma City, Oklahoma, United States",35.472989,-97.517054,"Oklahoma City, Oklahoma County, Oklahoma, Unit..."


In [57]:
export_coordinates_concat(df_coordinates)

df_coordinates_scraped: (1790, 7)

Found 6 duplicates:
               city         country
673      Nottingham  United Kingdom
1283    Los Angeles   United States
1397       New York   United States
1406  New York City   United States
1480   Philadelphia   United States
1618       Scranton   United States

Resulting dataset: (1798, 7)
Merged artists with coordinates! Found 8 new locations
df_coordinates_concat exported to .csv


In [58]:
# merge with the dataframe containing all the artists and their origins
df_artists_origins_coordinates_concat = merge_origins_coordinates(df_new_artists)
df_artists_origins_coordinates_concat.tail()

Exported to a .csv file


,artist,country,city,latitude,longitude,address
3976,Kingdom of Giants,United States,Sacramento,38.581061,-121.493895,"Sacramento, Sacramento County, California, Uni..."
3977,Guitar Fight from Fooly Cooly,United States,Cleveland,35.159518,-84.876611,"Cleveland, Bradley County, East Tennessee, Ten..."
3978,Black Stone Cherry,United States,Edmonton,36.980056,-85.612191,"Edmonton, Metcalfe County, Kentucky, 42129, Un..."
3979,Nothing,United States,Philadelphia,39.952724,-75.163526,"Philadelphia, Philadelphia County, Pennsylvani..."
3980,The Fall of Troy,United States,Mukilteo,47.947403,-122.303699,"Mukilteo, Snohomish County, Washington, United..."


## **Statistics**

In [59]:
df_artists_origins_coordinates_concat = pd.read_csv('Datasets/df_artists_origins_coordinates.csv')
df_artists_origins_coordinates_concat.shape

(3981, 6)

In [60]:
brittish_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United Kingdom']
american_cities = df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']

In [61]:
order = df_artists_origins_coordinates_concat['country'].value_counts().index
print(df_artists_origins_coordinates_concat['country'].value_counts().head())

# plt.figure(figsize=(8,6))
# sns.countplot(df_artists_origins_coordinates_concat['country'], order=order)

country
United States     2046
United Kingdom    1299
Canada             123
Sverige             93
Australia           90
Name: count, dtype: int64


In [111]:
print(f"{brittish_cities.shape[0]} Brittish artists")
order = brittish_cities['city'].value_counts().index
print(f"{brittish_cities['city'].nunique()} Brittish cities\n")
print(brittish_cities['city'].value_counts().head(15))

# plt.figure(figsize=(9,45))
# sns.countplot(brittish_cities['city'], order=order)

1297 Brittish artists
397 Brittish cities

city
London                 348
Glasgow                 39
Birmingham              36
Manchester              32
Leeds                   31
Brighton                27
Liverpool               26
Bristol                 23
Sheffield               17
Edinburgh               14
Cardiff                 14
Nottingham              13
Southampton             11
Reading                 11
Newcastle upon Tyne     10
Name: count, dtype: int64


In [112]:
print(f"{american_cities.shape[0]} American artists")
order = american_cities['city'].value_counts().index
print(f"{american_cities['city'].nunique()} American cities\n")
print(american_cities['city'].value_counts().head(15))

# plt.figure(figsize=(5,55))
# sns.countplot(df_artists_origins_coordinates_concat[df_artists_origins_coordinates_concat['country']=='United States']['city'], order=order)

2034 American artists
582 American cities

city
Los Angeles      220
New York City    133
Chicago           67
San Francisco     63
Seattle           51
Brooklyn          46
Boston            38
Philadelphia      33
Portland          28
San Diego         24
Washington        23
Austin            22
Phoenix           22
Detroit           20
New York          18
Name: count, dtype: int64
